# Prosta agregacija (engl. bagging)

Stabla odlučivanja pripadaju posebnoj klasi modela proste agregacije. Ovi modeli se zansivaju na ideji obučavanja više istih modela nad proizvoljnim podskupovima podataka i njihovih atributa sa ciljem da se redukuje varijabilnost samih modela. Konačna odluka modela se u slučaju regresionih zadataka dobija uprosečavanjem predikcija pojedinačnih modela, dok se u slučaju klasifikacionih zadataka o finalnom obeležju odlučuje na osnovu većinskog glasanja. 

Primer korišćenja omotača koji omogućava kreiranje ansambala prostom agregacijom videćemo na primeru klasifikacije i skupu podataka za klasifikaciju tumora na benigne i maligne. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn import datasets
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics

from sklearn import ensemble
from sklearn import tree
from sklearn import neighbors

Prvo ćemo učitati podatke i pripremiti ih za dalji rad.

In [3]:
data = datasets.load_breast_cancer()

In [4]:
# print(data.DESCR)

In [5]:
X = data.data
y = data.target

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)

In [7]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train, y_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Za kreiranje proste agregacije može se koristi `BaggingClassifier` metoda `ensemble` paketa. Ona od parametara očekuje
osnovni model, broj estimatora koji treba kreirati (parametar `n_estimators`), maksimalni broj instanci koje treba uzeti u podskupu (parametar `max_samples`), kao i maksimalan broj atributa na nivou instanci (parametar `max_features`). Kreiranje slučajnih podskupova atributa i instanci se može kombinovati ili izvoditi pojedinačno. Posebno, parametrima `booststrap` i `bootstrap_features` se kontroliše da li se ovi slučajni podskupovi generišu sa ili bez ponavljanja. Praćenje  slučajnosti se vrši preko parametra `random_state`.

<table>
<tr>
    <td><img src='assets/with_replacement.png' style='width: 400px;'> </td>
    <td> <img src='assets/without_replacement.png' style='width: 400px;'></td>
<tr>
</table>

In [8]:
model_tree = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(random_state=42), n_estimators=500, max_samples=100, bootstrap=True, random_state=42)

In [9]:
model_tree.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=42,
  

In [10]:
y_predicted = model_tree.predict(X_test)

In [11]:
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91        70
           1       0.95      0.94      0.94       118

    accuracy                           0.93       188
   macro avg       0.93      0.93      0.93       188
weighted avg       0.93      0.93      0.93       188



Za bazni klasifikator se može iskoristiti proizvoljni klasifikator, npr. KNN. 

In [12]:
model_knn = ensemble.BaggingClassifier(neighbors.KNeighborsClassifier(n_neighbors=5, metric='cosine'), n_estimators=100, max_samples=0.9, bootstrap=False, random_state=42)

In [13]:
model_knn.fit(X_train, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cosine',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=5, p=2,
                                                      weights='uniform'),
                  bootstrap=False, bootstrap_features=False, max_features=1.0,
                  max_samples=0.9, n_estimators=100, n_jobs=None,
                  oob_score=False, random_state=42, verbose=0,
                  warm_start=False)

In [14]:
y_predicted = model_knn.predict(X_test)

In [15]:
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95        70
           1       0.96      0.98      0.97       118

    accuracy                           0.96       188
   macro avg       0.96      0.96      0.96       188
weighted avg       0.96      0.96      0.96       188

